Installations done before
    
    pip3 install pytorch-pretrained-bert
    pip3 install pytorch-nlp
    pip3 install transformers

### Imports

In [1]:
import json
import os
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score
import torch
from transformers import BertTokenizer, BertModel

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
from utils import HotPotDataHandler

### Encoder

In [3]:
def encode(text,
           tokenizer=BertTokenizer.from_pretrained('bert-base-uncased'),
           model=BertModel.from_pretrained('bert-base-uncased',
                                 output_hidden_states=True,
                                 output_attentions=True)):
    ''' TODO: document
    '''
    
    input_ids = torch.tensor([tokenizer.encode(text)])
    all_hidden_states, all_attentions = model(input_ids)[-2:]
    
    # This is the embedding of the [CLS] token.
    # [-1] is the last hidden state (list of sentences)
    # first [0] - first (and only) sentence
    # second [0] - first ([CLS]) token of the sentence
    return all_hidden_states[-1][0][0]
    

### Paragraph Selector Class

In [10]:
"""
This module implements the Paragraph Selector from the paper, Section 3.1
"""

import torch
from transformers import BertTokenizer, BertModel

class ParagraphSelector():
    """
    TODO: write docstring
    """
    
    def __init__(self,
                 model_path=None,
                 tokenizer=None,
                 model=None):
        """
        TODO: write docstring
        """
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') if not tokenizer else tokenizer
        self.model = BertModel.from_pretrained('bert-base-uncased',
                                 output_hidden_states=True,
                                 output_attentions=True) if not model else model
        
        class ParagraphSelectorNet(torch.nn.Module):
            """
            TODO: write docstring
            """
            def __init__(self, input_size=768, output_size=1):
                super(ParagraphSelectorNet, self).__init__()
                self.linear  = torch.nn.Linear(input_size, output_size)

            def forward(self, embedding):
                output = self.linear(embedding)
                output = torch.sigmoid(output)

                return output 
            
        self.net = ParagraphSelectorNet()
        if model_path:
            try:
                self.net.load_state_dict(torch.load(model_path))
            except FileNotFoundError as e:
                print(e, model_path)
    
    def encode(self, text):
        ''' TODO: document
        '''

        input_ids = torch.tensor([self.tokenizer.encode(text)])
        all_hidden_states, all_attentions = self.model(input_ids)[-2:]

        # This is the embedding of the [CLS] token.
        # [-1] is the last hidden state (list of sentences)
        # first [0] - first (and only) sentence
        # second [0] - first ([CLS]) token of the sentence
        return all_hidden_states[-1][0][0]

    def train(self, train_data, labels, epochs, learning_rate=0.0001):
        """
        TODO: write docstring
        """
        # Use Binary Cross Entropy as a loss function instead of MSE
        # There are papers on why MSE is bad for classification
        criterion = torch.nn.BCELoss()
        optimizer = torch.optim.Adam(self.net.parameters(), lr=learning_rate)

        losses = []
        
        # Set the network into train mode
        self.net.train()

        print("Training...")

        # Iterate over the epochs
        for epoch in range(epochs):
            print('Epoch %d/%d' % (epoch + 1, epochs))
            for inputs, label in zip(train_data, labels):

                optimizer.zero_grad()
                outputs = self.net(inputs)
                loss = criterion(outputs, label)
                loss.backward(retain_graph=True)
                losses.append(loss.item())
                optimizer.step()

        return losses

    def predict(self, p):
        """
        TODO: write docstring
        """
        self.net.eval()
        score = self.net(p)
        return score

    def evaluate(self, data, threshold=0.1):
        """
        TODO: write docstring
        """
        all_true = []
        all_pred = []
        ids = []
        
        for point in data:
            context = self.make_context(point, threshold) #point[2] are the paragraphs, point[1] is the query
            para_true = []
            para_pred = []
            for para in point[3]:
                para_true.append(para[0] in point[1])
                para_pred.append(para in context)
            all_true.append(para_true)
            all_pred.append(para_pred)
            ids.append(point[0])
        
        # Flatten the lists so they can be passed to the precision and recall funtions
        all_true_flattened = [point for para in all_true for point in para]
        all_pred_flattened = [point for para in all_pred for point in para]
        
        precision = precision_score(all_true_flattened, all_pred_flattened)
        recall = recall_score(all_true_flattened, all_pred_flattened)
        
        return precision, recall, ids, all_true, all_pred
    
    def make_context(self, datapoint, threshold=0.1):
        """
        TODO: write docstring
        
        Parameters: paragraphs - [[p1_title, [p1_s1, p1_s2 ...]],
                                  [p2_title, [p2_s1, p2_s2, ...]],
                                   ...]
                    query - the query as a string
                    threshold - a float between zero and one;
                                paragraphs that get a score above the
                                threshold, become part of the context
        Output: context: [[p1_title, [p1_s1, p1_s2 ...]],
                          [p2_title, [p2_s1, p2_s2, ...]],
                           ...]
        """
        context = []
        for p in datapoint[3]:
            # p[0] is the paragraph title, p[1] is the list of sentences in the paragraph
            encoded_p = self.encode("[CLS] " + datapoint[2] + " [SEP] " + ("").join(p[1]) + " [SEP]")
            score = self.predict(encoded_p)
            if score > threshold:
                context.append(p)
        return context
    
    def save(self, savepath):
        '''
        TODO: write docstring
        '''
        directory_name = "/".join(savepath.split('/')[:-1])
        print(directory_name)
        if not os.path.exists(directory_name):
            os.makedirs(directory_name)
        torch.save(self.net.state_dict(), savepath)


In [11]:
dh = HotPotDataHandler(parent_dir + "/data/hotpot_train_v1.1.json")

In [12]:
data = dh.data_for_paragraph_selector()

In [13]:
# data[:1]

In [14]:
def make_training_data(data,
                       tokenizer=BertTokenizer.from_pretrained('bert-base-uncased'),
                       model=BertModel.from_pretrained('bert-base-uncased',
                                 output_hidden_states=True,
                                 output_attentions=True)):
    '''
    Make a dataframe with training data for selecting relevant paragraphs
    Each entry in the dataframe has three columns:
        1. Query - the question
        2. Paragraphs - the paragraphs
        3. Label - 0 (unrelated) or 1 (related)
    '''
    labels = []
    datapoints = []
    for point in data:        
        for para in point[3]:
            labels.append(torch.Tensor([int(para[0] in point[1])])) # Label 1: if paragraph title is in supporting facts, otherwise 0
            encoded_point = encode("[CLS] " + point[2] + " [SEP] " + ("").join(para[1]) + " [SEP]", tokenizer, model)
            datapoints.append(encoded_point)
        
    df = pd.DataFrame({
        'id': range(len(labels)),
        'label': labels,
        'text': datapoints
    })
    return df   

In [15]:
print("Reading data...")
dh = HotPotDataHandler(parent_dir + "/data/hotpot_train_v1.1.json")
data = dh.data_for_paragraph_selector()

print("Splitting data...")
training_data_raw, test_data_raw = train_test_split(data[:4], test_size=0.25, random_state=42, shuffle=True)
training_data = make_training_data(training_data_raw)
training_data = shuffle(training_data, random_state=42)

print("Initilising ParagraphSelector...")
ps = ParagraphSelector()
losses = ps.train(training_data["text"], training_data["label"], 1)

print("Evaluating...")
precision, recall, ids, y_true, y_pred = ps.evaluate(test_data_raw)
print("Precision:", precision)
print("Recall:", recall)

Reading data...
Splitting data...
Initilising ParagraphSelector...
Training...
Epoch 1/1
Evaluating...
Precision: 0.2
Recall: 1.0


In [15]:
training_data

,id,label,text
27,27,[tensor(0.)],"[tensor(-0.0142, grad_fn=<SelectBackward>), te..."
15,15,[tensor(1.)],"[tensor(-0.4818, grad_fn=<SelectBackward>), te..."
23,23,[tensor(1.)],"[tensor(-0.0365, grad_fn=<SelectBackward>), te..."
17,17,[tensor(1.)],"[tensor(-0.4297, grad_fn=<SelectBackward>), te..."
8,8,[tensor(0.)],"[tensor(-0.8046, grad_fn=<SelectBackward>), te..."
9,9,[tensor(0.)],"[tensor(-0.5943, grad_fn=<SelectBackward>), te..."
28,28,[tensor(0.)],"[tensor(-0.3586, grad_fn=<SelectBackward>), te..."
24,24,[tensor(1.)],"[tensor(-0.3114, grad_fn=<SelectBackward>), te..."
12,12,[tensor(0.)],"[tensor(-0.1670, grad_fn=<SelectBackward>), te..."
0,0,[tensor(0.)],"[tensor(-0.1817, grad_fn=<SelectBackward>), te..."


In [14]:
training_data_raw

[(['Peggy Seeger', 'Peggy Seeger', 'Ewan MacColl'],
  " What nationality was James Henry Miller's wife?",
  [['Moloch: or, This Gentile World',
    ['Moloch: or, This Gentile World is a semi-autobiographical novel written by Henry Miller in 1927-28, initially under the guise of a novel written by his wife, June.',
     ' The book went unpublished until 1992, 65 years after it was written and 12 years after Miller’s death.',
     ' It is widely considered to be of interest more as a study of Miller’s artistic growth than as a worthy piece of fiction.']],
   ['Launceston by-election, 1874',
    ['The Launceston by-election of 1874 was fought on 3 July 1874.',
     ' The byelection was fought due to the void Election of the incumbent Conservative MP, James Henry Deakin (senior).',
     ' It was won by the Conservative candidate James Henry Deakin (junior).']],
   ['Incest: From a Journal of Love',
    ['Incest: From a Journal of Love: The Unexpurgated Diary of Anaïs Nin (1932–1934) is a 1

In [16]:
test_data_rawrom_file = ParagraphSelector(model_path=parent_dir + '/models/paragraphSelector_2.pt')